# 2025 Symbolic regression Monod paper
# Pretreating real data and Generating simulated data

In [ ]:
cwd = "/scratch/project_2000746/anthosun/2025SRMO/"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

## Simple functions

In [ ]:
# smoothing

halfwindow = 3

def summingaround(N, halfwindow = 3):

    AA = N.copy()
    
    for w in np.arange(halfwindow) + 1: # summing over (2*halfwindow)-window
        BB = np.roll(AA, shift=-w, axis=0)
        BB[-w:] = 0
        AA = AA + BB
        BB = np.roll(AA, shift=w, axis=0)
        BB[:w] = 0
        AA = AA + BB
    
    return AA


def smoothing(N, halfwindow = 3):
    
    AA = summingaround(N, halfwindow=halfwindow)
    BB = np.ones(AA.shape)
    BB = summingaround(BB, halfwindow=halfwindow)
    
    return AA / BB

In [ ]:
# Compute per-capita derivatives

def to_rhos(T, N):
    
    if len(T.shape) == 1:
        assert T.shape[0] == N.shape[0], "T-array has length {} but N-array has length {}!".format(T.shape[0], N.shape[0])
        for axis in N.shape[1:]:
            T = T[...,np.newaxis]
    else:
        assert T.shape == N.shape, "T-array has shape {} but N-array has shape {}!".format(T.shape, N.shape)
    
    if T.dtype == int:
        T = T.astype(float)
    
    # delta N / N
    
    # forward roll for N
    A = np.roll(N, shift=-1, axis=0)
    A[-1:] = np.nan
    A = A - N
    A[np.isnan(A)] = 0

    # backward roll for N
    B = np.roll(N, shift=1, axis=0)
    B[:1] = np.nan
    B = B - N
    B[np.isnan(B)] = 0
    
    NNN = N.copy()
    NNN[N!=0] = (A[N!=0] - B[N!=0]) / N[N!=0]

    # delta T
    
    # forward roll for T
    A = np.roll(T, shift=-1, axis=0)
    A[-1:] = np.nan
    A = A - T
    A[np.isnan(A)] = 0

    # backward roll for T
    B = np.roll(T, shift=1, axis=0)
    B[:1] = np.nan
    B = B - T
    B[np.isnan(B)] = 0

    T = A - B

    return NNN / T

In [ ]:
# Compute AUCs/integrals
# returns an array of same dimensions,
# with first time point being zero

def to_AUCs(T, N, t0 = 0 # t0: if integer, reference index for integral computation, otherwise the raw AUC is returned
           ):
    
    if len(T.shape) == 1:
        assert T.shape[0] == N.shape[0], "T-array has length {} but N-array has length {}!".format(T.shape[0], N.shape[0])
        for axis in N.shape[1:]:
            T = T[...,np.newaxis]
    else:
        assert T.shape == N.shape, "T-array has shape {} but N-array has shape {}!".format(T.shape, N.shape)
    
    A = ( np.roll(N, shift=1, axis=0) + N ) / 2 # average N
#    A[0] = N[0] # not useful since replaced immediately

    B = T - np.roll(T, shift=1, axis=0) # delta T
    
    A *= B # average N * delta T
    A[0] = 0 # cut off first value, # Flooring is not required when using trapezoidal rule
    
    B = np.cumsum(A, axis=0) # cumulative sum
    
    return B

# Experimental data

## Pretreat real data

In [ ]:
from os import listdir

for name in listdir("{}/raws".format(cwd)):

    if name[:3] != "Rs0":
        continue

    df = pd.read_csv("{}/raws/{}".format(cwd, name), index_col=[0,1] )
    data = pd.DataFrame()
    concs = sorted(list(df.index.get_level_values("conc").unique()))
    
    for conc in concs:
        
        N = df.xs(conc, level="conc")
        N = N.sort_index()

        T = np.array(N.index, dtype=float) / 60 / 60 # convert time unit from seconds to hours
        N = np.mean(N.to_numpy(), axis=1) # mean across replicates
        N = smoothing(N) # smoothing
        Nc = to_AUCs(T, N, t0=0)
        F = np.exp(-Nc)
        R = to_rhos(T, N)

        dataframe = pd.DataFrame()
        dataframe["T"] = T
        dataframe["n"] = N
        dataframe["Nc"] = Nc
        dataframe["F"] = F
        dataframe["U"] = np.exp(- T)
        dataframe["rho"] = R
        dataframe["C"] = conc

        data = pd.concat([data, dataframe])
        
    data.to_csv("{}/data/{}".format(cwd, name), sep=",", index=False)

## Visualisation Fig.S1

In [ ]:
code = "R"
resc = "0"

freqorder = list(pd.read_csv("/scratch/project_2000746/anthosun/2025SRMO/raws/Frequencies.csv", sep=",", index_col=0,).index)

In [ ]:
nplots = 2 # nb of mini-plots per strain
nranks = 2 # nb of meta-columns

YYY = len(freqorder) // nranks
XXX = nplots * nranks

cmap = mpl.colormaps["viridis"]

In [ ]:
fig = plt.figure(figsize=(10,8), constrained_layout=True)
fig.supxlabel("time")
fig.supylabel("bacterial species (HAMBI code)")

gs = fig.add_gridspec(YYY, XXX) # (y, x)
axs = gs.subplots()

for i, strn in enumerate(freqorder): # go through strains
    strn = str(strn)
    rank = i // YYY
    row = i % YYY
    
    data = pd.read_csv("{}/data/{}s{}i{}.csv".format(cwd, code, resc, strn), sep=",")
    concs = sorted(list(data["C"].apply(float).unique()))

    for k, y in enumerate(["n", "rho"]):
        ax = axs[row,nplots*rank+k]
        
        for c, conc in enumerate(concs):
            colour = cmap(c / len(concs))
    
            datc = data[data["C"] == conc]
            xs = [datc["T"].to_numpy(), datc["T"].to_numpy(), np.arange(0, 5000, 1)]
            ax.plot(xs[k], datc[y], c=colour)
            
        ax.tick_params(axis="both", which="both", bottom=False, top=False, left=False, right=False, labelbottom=False, labeltop=False, labelleft=False, labelright=False, )
        ax.set(xlim = (0, np.max(xs[k])), #ylim = (0, data[y].max() * 1.1),
               ylabel = str(strn) if k == 0 else None,
               title = [r"population size $N$", r"per capita growth rate $\rho_{\rm obs}$", ][k] if row == 0 else None, )

### SAVING
name = "Time_dependence_{}s{}.png".format(code, resc)
plt.savefig("{}/plot/{}".format(cwd, name), facecolor='w', edgecolor='w', transparent=False, bbox_inches="tight")
plt.show()

# Simulated data

## Load simulation parameters from /cwd/pars

In [ ]:
concs = np.genfromtxt("{}/pars/concs.csv".format(cwd,), delimiter=",")
nuxi = np.genfromtxt("{}/pars/nuxi.csv".format(cwd,), delimiter=",")
K_ix = np.genfromtxt("{}/pars/K_ix.csv".format(cwd,), delimiter=",")
gmix = np.genfromtxt("{}/pars/gmix.csv".format(cwd,), delimiter=",")
qm_i = np.genfromtxt("{}/pars/qm_i.csv".format(cwd,), delimiter=",")

In [ ]:
nconcs = len(concs)
nrescs, nstrns = nuxi.shape

assert K_ix.shape == (nstrns, nrescs)
assert gmix.shape == (nstrns, nrescs)
assert qm_i.shape == (2, nstrns)

rescs = np.arange(nrescs) + 1
strns = np.arange(nstrns)

## Dynamical system

In [ ]:
# resource-consumer model for simulation

def RCM(t, X, substrate_nb, growth_function, nuxi, *args):
    
    S, N = X[:substrate_nb], X[substrate_nb:]
    dX = np.zeros(X.shape)
    dX[:substrate_nb] = (nuxi @ N) * S # dynamics of the substrates
    dX[substrate_nb:] = growth_function(S, *args) * N # dynamics of the population

    return dX

In [ ]:
# adjustment function

def alpha_RCM(t, q, m):
    
    alpha = q / (q + np.exp(- m * t))

    return alpha

## Growth functions

In [ ]:
# model A (additive with global g_max)

def fA(S,    # (        nrescs)
       gmix, # (nstrns, nrescs)
       K_ix, # (nstrns, nrescs)
      ):
    
    F = S[np.newaxis,:] / ( K_ix + S[np.newaxis,:] ) # (nstrns, nrescs)
    F = gmix[:,0] * np.sum(F, axis = -1) # (nstrns)
    
    return F # (nstrns)

In [ ]:
# model M (multiplicative)

def fM(S, gmix, K_ix):
    
    F = S[np.newaxis,:] / ( K_ix + S[np.newaxis,:] )
    F = gmix[:,0] * np.prod(F, axis = -1)
    
    return F

# Simulation and saving

In [ ]:
# simulation time

dt = 0.1
tmax = 50000
subtime = 500 # timepoint subsampling: 1/500

ts = np.arange(0, tmax + dt, dt) # timepoints for simulation
subts = ts[::subtime,np.newaxis]
T = np.repeat(subts, nconcs, axis=-1) # # timepoints for sub-sampling (ntimes, nconcs)
N = np.zeros(T.shape) # sub-sampled population sizes (ntimes, nconcs)

ntimes = ts.shape[0]
nsbtms = T.shape[0]

In [ ]:
# simulation

for resc in rescs:
    
    nu = nuxi[:resc,:]
    gm = gmix[:,:resc]
    K_ = K_ix[:,:resc]
    
    for modl in [fA, fM]:
        for strn in strns[:1]:
                
            name = "{}s{}i{}.csv".format(modl.__name__[1], resc, strn)
            
            for c, conc in enumerate(concs):
                print(name, c, "   ", end="\r")
                
                # intial condition
                S0 = np.ones(resc) * 0.1 * conc #/ (resc)
                N0 = np.zeros(nstrns)
                N0[strn] = 0.1
                X0 = np.concatenate([S0, N0])
                
                # Euler's method
                for t, time in enumerate(ts):
                    dX = RCM(t, X0, resc, modl, nu, gm, K_,) * dt # Euler step
                    dX[resc:] *= alpha_RCM(time, qm_i[0], qm_i[1]) # adjustment function alpha
                    X0 += dX
                    # record only if the time is to be kept in subtimes Ts
                    if time in subts:
                        N[np.argmax(subts==time),c] = np.sum(X0[resc:]) # (ntimes, nconcs)
            
            Nc = to_AUCs(T, N, t0=0)
            F = np.exp(-Nc)
            R = to_rhos(T, N)
            
            df = pd.DataFrame()
            df["t"] =   T.flatten(order="F")
            df["N"] =   N.flatten(order="F")
            df["Nc"] = Nc.flatten(order="F")
            df["F"] =   F.flatten(order="F")
            df["rho"] = R.flatten(order="F")
            df["C"] = np.repeat(concs, nsbtms)
            
            df.to_csv("{}/data/{}".format(cwd, name), sep=",", index=False)

# Add lfit-inferred adjustment function alpha as a new column

In [ ]:
from os import listdir

files = sorted(listdir("{}/data".format(cwd)))
lfit_results = pd.read_csv("{}/lfit/lfit_results.csv".format(cwd), sep=",", header=[0,1,2,3], index_col=[0], )

In [ ]:
for file in files:
    
    name = file[:file.index(".csv")]
    code = name[:name.index("s")]
    resc = name[name.index("s")+1:name.index("i")]
    strn = name[name.index("i")+1:]
    
    data = pd.read_csv("{}/data/{}".format(cwd, file), sep=",")
    data["inferred_alpha"] = data["C"].apply(lambda x: lfit_results.loc["q",(code,resc,strn,str(x))] ) # parameter q
    data["m"] = data["C"].apply(lambda x: lfit_results.loc["m",(code,resc,strn,str(x))] )
    data["inferred_alpha"] = data["inferred_alpha"] / (np.exp(- data["m"] * data["t"]) + data["inferred_alpha"] )
    data.drop("m", axis=1, inplace=True)

    data.to_csv("{}/data/{}".format(cwd, file), sep=",", index=False)
    print(name, "   ", end="\r")

# Add lfit-inferred single adjustment function alpha
# (one across all concentrations) as a new column

In [ ]:
from os import listdir

files = sorted(listdir("{}/data".format(cwd)))
lfit_results = pd.read_csv("{}/lfit/lfit_results_single_alpha.csv".format(cwd), index_col=[0], header=[0,1,2])

In [ ]:
for file in files:
    
    name = file[:file.index(".csv")]
    code = name[:name.index("s")]
    resc = name[name.index("s")+1:name.index("i")]
    strn = name[name.index("i")+1:]
    
    data = pd.read_csv("{}/data/{}".format(cwd, file), sep=",")
    q = lfit_results.loc["q",(code,resc,strn)] # parameter q
    m = lfit_results.loc["m",(code,resc,strn)] # parameter q
    data["inferred_single_alpha"] = q / (np.exp(- m * data["t"]) + q )

    data.to_csv("{}/data/{}".format(cwd, file), sep=",", index=False)
    print(name, "   ", end="\r")

# Simulation and saving of no-adjustment data

In [ ]:
# simulation time

dt = 0.1
tmax = 50000
subtime = 500 # timepoint subsampling: 1/500

ts = np.arange(0, tmax + dt, dt) # timepoints for simulation
subts = ts[::subtime,np.newaxis]
T = np.repeat(subts, nconcs, axis=-1) # # timepoints for sub-sampling (ntimes, nconcs)
N = np.zeros(T.shape) # sub-sampled population sizes (ntimes, nconcs)

ntimes = ts.shape[0]
nsbtms = T.shape[0]

In [ ]:
# simulation

for resc in rescs[-1:]:
    
    nu = nuxi[:resc,:]
    gm = gmix[:,:resc]
    K_ = K_ix[:,:resc]
    
    for modl in [fA, fM][-1:]:
        for strn in strns:
                
            name = "{}s{}i{}.csv".format(modl.__name__[1], resc, strn)
            
            for c, conc in enumerate(concs):
                print(name, c, "   ", end="\r")
                
                # intial condition
                S0 = np.ones(resc) * 0.1 * conc #/ (resc)
                N0 = np.zeros(nstrns)
                N0[strn] = 0.1
                X0 = np.concatenate([S0, N0])
                
                # Euler's method
                for t, time in enumerate(ts):
                    dX = RCM(t, X0, resc, modl, nu, gm, K_,) * dt # Euler step
#                    dX[resc:] *= alpha_RCM(time, qm_i[0], qm_i[1]) # adjustment function alpha
                    X0 += dX
                    # record only if the time is to be kept in subtimes Ts
                    if time in subts:
                        N[np.argmax(subts==time),c] = np.sum(X0[resc:]) # (ntimes, nconcs)
            
            Nc = to_AUCs(T, N, t0=0)
            F = np.exp(-Nc)
            R = to_rhos(T, N)
            
            df = pd.DataFrame()
            df["t"] =   T.flatten(order="F")
            df["N"] =   N.flatten(order="F")
            df["Nc"] = Nc.flatten(order="F")
            df["F"] =   F.flatten(order="F")
            df["rho"] = R.flatten(order="F")
            df["C"] = np.repeat(concs, nsbtms)
            
            df.to_csv("{}/cplm/data/{}".format(cwd, name), sep=",", index=False)

## Simulation and saving of gLV data with and without adjustment

In [ ]:
# resource-consumer model for simulation

def gLV(t, X, substrate_nb, gmix, Kix):
    
    S, N = X[:substrate_nb], X[substrate_nb:]
    dX = np.zeros(X.shape)
    dX[substrate_nb:] = gmix[:,0] * (S[0] * K_ix[:,0] - N) * N # dynamics of the population

    return dX

In [ ]:
# simulation time

dt = 0.1
tmax = 50000
subtime = 500 # timepoint subsampling: 1/500

ts = np.arange(0, tmax + dt, dt) # timepoints for simulation
subts = ts[::subtime,np.newaxis]
T = np.repeat(subts, nconcs, axis=-1) # # timepoints for sub-sampling (ntimes, nconcs)
N = np.zeros(T.shape) # sub-sampled population sizes (ntimes, nconcs)

ntimes = ts.shape[0]
nsbtms = T.shape[0]

In [ ]:
# simulation

for resc in rescs[:1]:
    
    nu = nuxi[:resc,:]
    gm = gmix[:,:resc]
    K_ = K_ix[:,:resc]
    
    for strn in strns:
                
        name = "{}s{}i{}.csv".format("G", 0, strn)
            
        for c, conc in enumerate(concs):
            print(name, c, "   ", end="\r")
                
            # intial condition
            S0 = np.ones(resc) * 0.1 * conc #/ (resc)
            N0 = np.zeros(nstrns)
            N0[strn] = 0.1
            X0 = np.concatenate([S0, N0])
                
            # Euler's method
            for t, time in enumerate(ts):
                dX = gLV(t, X0, resc, gm, K_,) * dt # Euler step
                dX[resc:] *= alpha_RCM(time, qm_i[0], qm_i[1]) # adjustment function alpha
                X0 += dX
                # record only if the time is to be kept in subtimes Ts
                if time in subts:
                    N[np.argmax(subts==time),c] = np.sum(X0[resc:]) # (ntimes, nconcs)
        
        Nc = to_AUCs(T, N, t0=0)
        F = np.exp(-Nc)
        R = to_rhos(T, N)
        
        df = pd.DataFrame()
        df["t"] =   T.flatten(order="F")
        df["N"] =   N.flatten(order="F")
        df["Nc"] = Nc.flatten(order="F")
        df["F"] =   F.flatten(order="F")
        df["rho"] = R.flatten(order="F")
        df["C"] = np.repeat(concs, nsbtms)
        
        df.to_csv("{}/cplm/data/{}".format(cwd, name), sep=",", index=False)

In [ ]:
# simulation

for resc in rescs[:1]:
    
    nu = nuxi[:resc,:]
    gm = gmix[:,:resc]
    K_ = K_ix[:,:resc]
    
    for strn in strns:
                
        name = "{}s{}i{}.csv".format("V", 0, strn)
            
        for c, conc in enumerate(concs):
            print(name, c, "   ", end="\r")
                
            # intial condition
            S0 = np.ones(resc) * 0.1 * conc #/ (resc)
            N0 = np.zeros(nstrns)
            N0[strn] = 0.1
            X0 = np.concatenate([S0, N0])
                
            # Euler's method
            for t, time in enumerate(ts):
                dX = gLV(t, X0, resc, gm, K_,) * dt # Euler step
#                dX[resc:] *= alpha_RCM(time, qm_i[0], qm_i[1]) # adjustment function alpha
                X0 += dX
                # record only if the time is to be kept in subtimes Ts
                if time in subts:
                    N[np.argmax(subts==time),c] = np.sum(X0[resc:]) # (ntimes, nconcs)
        
        Nc = to_AUCs(T, N, t0=0)
        F = np.exp(-Nc)
        R = to_rhos(T, N)
        
        df = pd.DataFrame()
        df["t"] =   T.flatten(order="F")
        df["N"] =   N.flatten(order="F")
        df["Nc"] = Nc.flatten(order="F")
        df["F"] =   F.flatten(order="F")
        df["rho"] = R.flatten(order="F")
        df["C"] = np.repeat(concs, nsbtms)
        
        df.to_csv("{}/cplm/data/{}".format(cwd, name), sep=",", index=False)